In [7]:
import numpy as np
from scipy.special import eval_hermite, eval_genlaguerre, j1, genlaguerre
from scipy.optimize import newton
import imageio as iio
import os
import math

# Define Hermite polynomial
def hermite(n, x):
    return eval_hermite(n, x)

# 屏幕参数
# 激光参数
z = 0
lamda = 0.66e-6  # 单位微米
w0 = 900e-6
zr = np.pi * w0**2 / lamda
wz = w0 * np.sqrt(1 + (z / zr)**2)
k0 = 2 * np.pi / lamda
pix_size = 12.5e-6

mx = 1272
my = 1024
x = np.linspace(-mx * pix_size / 2, mx * pix_size / 2, mx)
y = np.linspace(-my * pix_size / 2, my * pix_size / 2, my)
X, Y = np.meshgrid(x, y)
r = np.sqrt(X**2 + Y**2)
theta = np.arctan2(Y, X)

# 平移参数
shift_x = 110 * pix_size
shift_y = 150 * pix_size
X_shifted = X + shift_x
Y_shifted = Y + shift_y

# Function to generate LG mode
def generate_LGf(X, Y, r, theta):
    l = 1
    p = 0
    L = abs(l)
    phase = np.exp(1j * l * theta)
    am = (2 * math.factorial(p) / (np.pi * math.factorial(p + L)))**0.5 * \
         (np.sqrt(2) * r / w0)**L * np.exp(-(r**2 / w0**2)) * \
         genlaguerre(p, L)(2 * r**2 / w0**2)
    return phase * am

uLG = generate_LGf(X_shifted, Y_shifted, np.sqrt(X_shifted**2 + Y_shifted**2), np.arctan2(Y_shifted, X_shifted))
AMLG = np.abs(uLG) / np.max(np.abs(uLG))
phiLG = np.angle(uLG)

# Function to generate HG mode
def HG_mode(nx, ny, X, Y, wx, wy):
    return hermite(nx, np.sqrt(2) * X / wx) * hermite(ny, np.sqrt(2) * Y / wy) * \
           np.exp(-(X**2 / wx**2 + Y**2 / wy**2))

uhg01 = HG_mode(0, 2, X_shifted, Y_shifted, wz, wz)
AMhg01 = np.abs(uhg01) / np.max(np.abs(uhg01))
phihg01 = np.angle(uhg01)

uhg10 = HG_mode(1, 0, X_shifted, Y_shifted, wz, wz)
AMhg10 = np.abs(uhg10) / np.max(np.abs(uhg10))
phihg10 = np.angle(uhg10)

# Function to generate Gaussian mode
def generate_GS(X, Y, r, theta):
    l = 0
    p = 0
    L = abs(l)
    phase = np.exp(-1j * l * theta)
    am = (2 * math.factorial(p) / (np.pi * math.factorial(p + L)))**0.5 * \
         (np.sqrt(2) * r / w0)**L * np.exp(-(r**2 / w0**2)) * \
         genlaguerre(p, L)(2 * r**2 / w0**2)
    return phase * am

ugs = generate_GS(X_shifted, Y_shifted, np.sqrt(X_shifted**2 + Y_shifted**2), np.arctan2(Y_shifted, X_shifted))
AMgs = np.abs(ugs) / np.max(np.abs(ugs))
phigs = np.angle(ugs)

# 旋转二维光栅
theta1 = np.radians(45)  # 45度旋转
theta2 = np.radians(135) # 135度旋转

# 生成旋转后的光栅
X_rot1 = X * np.cos(theta1) - Y * np.sin(theta1)
Y_rot1 = X * np.sin(theta1) + Y * np.cos(theta1)

X_rot2 = X * np.cos(theta2) - Y * np.sin(theta2)
Y_rot2 = X * np.sin(theta2) + Y * np.cos(theta2)
# Add grating phase
grating_period = pix_size * 5
graLG = (2 * np.pi * X / grating_period) % (2 * np.pi)
graHG01 = (2 * np.pi * Y / grating_period) % (2 * np.pi)
graHG10 = (2 * np.pi * X_rot1 / grating_period) % (2 * np.pi)
graGS = (2 * np.pi * X_rot2 / grating_period) % (2 * np.pi)

phiLG = phiLG + graLG - np.pi % (2 * np.pi)
phiHG01 = phihg01 + graLG - np.pi % (2 * np.pi)
phiHG10 = phihg10 + graLG - np.pi % (2 * np.pi)
phiGS = phigs + graLG - np.pi % (2 * np.pi)

# Method F
C = 0.5818

def faeq(f_a_ref, a_ref):
    global C
    return j1(f_a_ref) - 0.5818 * a_ref

def find_f_a_ref(a_ref):
    return newton(faeq, np.ones_like(a_ref) * 1, args=(a_ref,))

faLG = find_f_a_ref(AMLG)
hLG = faLG * np.sin(phiLG)

faHG01 = find_f_a_ref(AMhg01)
hHG01 = faHG01 * np.sin(phiHG01)

faHG10 = find_f_a_ref(AMhg10)
hHG10 = faHG10 * np.sin(phiHG10)

faGS = find_f_a_ref(AMgs)
hGS = faGS * np.sin(phiGS)

h = hHG01
h = np.mod(h + np.min(h), 2 * np.pi)

# Create a full black image and place the phase pattern in the center
gh = (h * 125 / (2 * np.pi)).astype(np.uint8)

# Output directory
output_dir = r'E:\桌面\wkx1\程序\python\投影测量'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Save the phase pattern image
lg_phase = gh
lg_image_path = os.path.join(output_dir, 'LG10.bmp')
iio.imwrite(lg_image_path, lg_phase)

print(f"Images saved to {output_dir}")


Images saved to E:\桌面\wkx1\程序\python\投影测量


In [ ]:
!pip install imageio